<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/topology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algebraische Topology**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

![Normed Vector Space](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/P1S2all.jpg/798px-P1S2all.jpg)

https://de.m.wikipedia.org/wiki/Algebraische_Topologie

https://en.m.wikipedia.org/wiki/Characterizations_of_the_category_of_topological_spaces

https://en.m.wikipedia.org/wiki/Category_of_topological_spaces

https://en.m.wikipedia.org/wiki/Topological_space

## **Differentiations**

#### **Measure Theory vs General Topology**

* Although there are superficial similarities, measure theory and general topology have quite different purposes.


* The aim in topology is to define structures that permit us to **define what we mean by a continuous function**. The most common way to do this is to **define a family of subsets of a set which we regard as being open**. The family of open sets must obey two axioms: the union of any set of open sets is open, the intersection of a finite collection of open sets is open. I'll also define a neighbourhood of a point as any superset of any open set containing the point. Then we can say **a function from a space X to a space Y is continuous at x if the inverse image of any neighbourhood of f(x) is a neighbourhood of x**. (Try to see how this generalises epsilon delta definitions.)

* The aim in measure theory is to **study measures of the size of a set such as length, area or volume (for example in vector spaces, or probability in probability spaces)**. Measures must obey certain axioms, the most important one being that the measure of the union of a finite number of disjoint sets is the sum of the measures. Mostly this is also extended to countable unions. It turns out that **it is not usually possible to define a measure on every subset so we define a class of measurable sets**. These obey superficially similar axioms to open sets. A countable union of measurable sets is measurable and the intersection and difference of two measurable sets is measurable. But notice the different restrictions on the number of sets. (It is quite possible, and frequently happens, that a measure is defined on a topological space. Vector spaces are an example.)

* **In measure theory there is a focus on σ-algebras**, i.e. non-empty subcollections of the powerset of a fixed set Ω that are closed under countable unions and complements. **In general topology there is a focus on topologies, i.e. subcollections of the powerset of a fixed set 𝑋 that are closed under finite intersections and arbitrary unions**. They “come together” if you are dealing with σ-algebras that are generated by topologies. Then we meet the **Borel σ-algebras**. If you study general topology then you can do completely without measure theory. If you study measure theory then some (not much) knowledge of topology is required. This especially when the Borel σ-algebras are on the menu.

* Measures deal with metrics and sets, which are commonly defined within a topology. General topology deals with properties of topological spaces, including the behavior of sets and sequences.

https://www.quora.com/What-are-the-main-similarities-and-differences-between-measure-theory-and-general-topology

#### **Geometry, Topology & Algebraic Topology**

**Geometry**

* The central object of study in plane geometry is congruence. Two shapes are congruent when one can be mapped to the other via a rigid motion: sliding it along, rotating it, or reflecting it. 

* No deformations, expansions, or other twists are allowed. So in geometry we can talk about angles, for example, since angles don't change when you slide and rotate. Congruent triangles are ones that are the same except for a possible translation (sliding), rotation and maybe reflection (taking a mirror image).

**Topology**

* First, we're not limited to simple polygonal shapes in the plane - we're dealing with any "shape" (topological space) for which we have a notion of "nearness". Intuitively it's fine to think of familiar shapes in the plane, space or (if your imagination is up to it) 4 dimensions and beyond. In fact, topology is a lot more permissive than this, and allows for spaces that look like nothing that fits into any number of dimensions.

* The second way topology differs from plane geometry is that the notion of "being the same thing", instead of congruence, is called topological invariance, which is formally defined as a bi-directional mapping of one shape to another which preserves this notion of points being near each other. It's ok to stretch and twist, it's ok to tear and paste back, but it's not ok to make a hole and keep it there, or to otherwise break things apart.

* If you have a rubber band, you can use it to form a circle, an ellipse, or a square. Those are very different shapes geometrically, but from the topological perspective they are the same.

**Algebraic Topology**

* in many ways the permissive nature of topology makes it hard to tell things apart. It's not entirely easy to show that the plane and 3d space are different topologically, which sounds a bit silly but it's a fact. Showing that 3d space isn't 4d space is even harder.

* One of the ways people found to deal with those difficulties is to create gadgets (officially called functors) that map topological spaces into objects that are easier to handle - algebraic objects like vector spaces and groups. If our layperson doesn't know what those are, we can get some headway by just declaring that our gadgets map topological spaces into numbers, or lists of numbers.

* We then show that if two topological spaces are the same (topologically), they will get the same numbers attached to them. Contrariwise, if two spaces are tagged with different numbers, they cannot be the same.

* Defining those gadgets isn't entirely trivial. Classically this was done via things called "homology" and "homotopy" groups. The latter ones can be roughly described as counting the number of distinct ways you can put a closed rubber band inside your space. Two such rubber bands aren't considered distinct if one can be deformed into the other inside your space.



## **Properties of Topological Vector Spaces**

### **Overview**

The definition of a topological space relies only upon set theory and is the most general notion of a mathematical space that allows for the definition of concepts such as 

* [continuity](https://en.m.wikipedia.org/wiki/Continuous_function#Continuous_functions_between_topological_spaces): intuitively, take nearby points to nearby points,

* [connectedness](https://en.m.wikipedia.org/wiki/Connected_space): 

* [convergence](https://en.m.wikipedia.org/wiki/Limit_of_a_sequence):

Other spaces, such as manifolds and **metric spaces, are specializations of topological spaces** with extra structures or constraints. 

Being so general, topological spaces are a central unifying notion and appear in virtually every branch of modern mathematics. The branch of mathematics that studies topological spaces in their own right is called [point-set topology](https://en.m.wikipedia.org/wiki/General_topology) or general topology.

https://en.m.wikipedia.org/wiki/Topological_space

https://de.m.wikipedia.org/wiki/Topologischer_Raum

**Point Set (General) Topology**

General topology is the branch of topology that deals with the basic set-theoretic definitions and constructions used in topology. It is the foundation of most other branches of topology, including differential topology, geometric topology, and algebraic topology. Another name for general topology is point-set topology.

The fundamental concepts in point-set topology are continuity, compactness, and connectedness:

* **Continuous functions**, intuitively, take nearby points to nearby points.

* **Compact sets** are those that can be covered by finitely many sets of arbitrarily small size.

* **Connected sets** are sets that cannot be divided into two pieces that are far apart.

https://en.m.wikipedia.org/wiki/General_topology

### **Lokalkonvexer Raum**

https://de.m.wikipedia.org/wiki/Lokalkonvexer_Raum

### **Topologischer Raum vs Topologischer Vektorraum**

https://de.m.wikipedia.org/wiki/Topologischer_Vektorraum

https://de.m.wikipedia.org/wiki/Topologischer_Raum

## **Invariants of Topological Spaces**

https://en.m.wikipedia.org/wiki/Fundamental_group

### **Topologische Invarianten (Properties)**

1. Anzahl der Zusammenhangs-Komponenten

2. Isomorphie-Klasse der Fundamentalgruppe

3. Euler-Charakteristik

4. Geschlecht einer Fläche

5. Bettizahlen eines topologischen Raumes

https://de.m.wikipedia.org/wiki/Topologische_Invariante

https://en.m.wikipedia.org/wiki/Topological_property

https://de.m.wikipedia.org/wiki/Geometrisierung_von_3-Mannigfaltigkeiten

https://de.m.wikipedia.org/wiki/Differentialtopologie

### **Homotopy Groups**

* Homotopy groups are used in algebraic topology to classify topological spaces. 

* The first and simplest homotopy group is the fundamental group, which records information about loops in a space. Intuitively, homotopy groups record information about the basic shape, or holes, of a topological space.

https://en.m.wikipedia.org/wiki/Homotopy_group

https://de.m.wikipedia.org/wiki/Homotopie

https://en.m.wikipedia.org/wiki/Regular_homotopy

https://en.m.wikipedia.org/wiki/Ambient_isotopy

### **Genus**

https://de.m.wikipedia.org/wiki/Geschlecht_(Fläche)

### **Fundamental Group**

* The fundamental group of a topological space is the group of the equivalence classes under homotopy of the loops contained in the space. It records information about the basic shape, or holes, of the topological space. 

* The fundamental group is the first and simplest homotopy group. The fundamental group is a homotopy invariant—topological spaces that are homotopy equivalent (or the stronger case of homeomorphic) have isomorphic fundamental groups.

### **Homology Groups**

Die Homologie ist ein mathematischer Ansatz, die Existenz von Löchern zu formalisieren. Gewisse „sehr feine“ Löcher sind für die Homologie unsichtbar; hier kann u. U. auf die schwerer zu bestimmenden Homotopiegruppen zurückgegriffen werden.



https://de.m.wikipedia.org/wiki/Homologietheorie

### **Triangulierung**

https://de.m.wikipedia.org/wiki/Triangulierung_(Topologie)

## **Classification of Topological Spaces**

#### **Trennungsaxiom**

https://de.m.wikipedia.org/wiki/Trennungsaxiom

https://en.m.wikipedia.org/wiki/History_of_the_separation_axioms

#### **Kolmogorov Space (T<sub>0</sub>)**

A space is Kolmogorov if for every pair of distinct points x and y in the space, there is at least either an open set containing x but not y, or an open set containing y but not x.

If for every pair of distinct points of X, at least one of them has a neighborhood not containing the other. In a T0 space, all points are topologically distinguishable.

https://en.m.wikipedia.org/wiki/Kolmogorov_space

#### **Fréchet Space (T<sub>1</sub>)**

Fréchet spaces are generalizations of Banach spaces (normed vector spaces that are complete with respect to the metric induced by the norm).

https://en.m.wikipedia.org/wiki/Fréchet_space

Instead, with the topology of compact convergence, C(a, b) can be given the structure of a Fréchet space: a locally convex topological vector space whose topology can be induced by a complete translation-invariant metric.

https://en.m.wikipedia.org/wiki/Locally_convex_topological_vector_space

#### **Haussdorf Spaces (T<sub>2</sub>)**

* A topological space is Hausdorff if and only if it is both preregular (i.e. topologically distinguishable points are separated by neighbourhoods) and Kolmogorov (i.e. distinct points are topologically distinguishable). A topological space is preregular if and only if its Kolmogorov quotient is Hausdorff.

* Almost all spaces encountered in analysis are Hausdorff; most importantly, the real numbers (under the standard metric topology on real numbers) are a Hausdorff space. More generally, all metric spaces are Hausdorff. In fact, many spaces of use in analysis, such as topological groups and topological manifolds, have the Hausdorff condition explicitly stated in their definitions.

https://en.m.wikipedia.org/wiki/Topological_vector_space

https://en.m.wikipedia.org/wiki/Hausdorff_space

## **Umgebung (Neighbourhood)**

* a neighbourhood (or neighborhood) is one of the basic concepts in a topological space. It is closely related to the concepts of open set and interior. 

* Intuitively speaking, a neighbourhood of a point is a set of points containing that point where one can move some amount in any direction away from that point without leaving the set.

https://de.m.wikipedia.org/wiki/Umgebung_(Mathematik)

https://en.m.wikipedia.org/wiki/Functional_analysis

https://en.m.wikipedia.org/wiki/Geometric_measure_theory